<a href="https://colab.research.google.com/github/lmcanavals/pcd/blob/master/0601_producers_consumers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt install spin

# Producers and Consumers

In [ ]:
%%file prodcons1.pml
#define wait(s) atomic { s > 0 -> s-- }
#define signal(s) s++

int buffer[5]
int n = 0

byte notEmpty = 0         // semaphore

inline append(d) {
    assert n < 5
    atomic {
        buffer[n] = d
        n++
    }
}

inline take(d) {
    assert n > 0
    atomic {
        n--
        d = buffer[n]
    }
}

active proctype Producer() {
    int d
    do
    ::
      // produce
      append(d)
      signal(notEmpty)
    od
}

active[5] proctype Consumer() {
    int d
    do
    ::
      wait(notEmpty)
      take(d)
      // consume
    od
}


Overwriting prodcons1.pml


In [ ]:
!spin prodcons1.pml

spin: prodcons1.pml:10, Error: assertion violated
spin: text of failed assertion: assert((n<5))
#processes: 6
		buffer[0] = 0
		buffer[1] = 0
		buffer[2] = 0
		buffer[3] = 0
		buffer[4] = 0
		n = 5
		notEmpty = 1
140295600:	proc  5 (Consumer:1) prodcons1.pml:19 (state 7)
140295600:	proc  4 (Consumer:1) prodcons1.pml:19 (state 7)
140295600:	proc  3 (Consumer:1) prodcons1.pml:19 (state 7)
140295600:	proc  2 (Consumer:1) prodcons1.pml:19 (state 7)
140295600:	proc  1 (Consumer:1) prodcons1.pml:37 (state 9)
140295600:	proc  0 (Producer:1) prodcons1.pml:27 (state 7)
6 processes created


In [ ]:
%%bash
rm *.trail pan.*
FILE=prodcons1.pml
spin -a $FILE && gcc pan.c && ./a.out
[ -f $FILE.trail ] && spin -t $FILE

pan:1: assertion violated (n<5) (at depth 181)
pan: wrote prodcons1.pml.trail

(Spin Version 6.4.6 -- 2 December 2016)
	+ Partial Order Reduction

Full statespace search for:
	never claim         	- (none specified)
	assertion violations	+
	acceptance   cycles 	- (not selected)
	invalid end states	+

State-vector 88 byte, depth reached 181, errors: 1
      182 states, stored
      256 states, matched
      438 transitions (= stored+matched)
        0 atomic steps
hash conflicts:         0 (resolved)

Stats on memory usage (in Megabytes):
    0.020	equivalent memory usage for states (stored*(State-vector + overhead))
    0.290	actual memory usage for states
  128.000	memory used for hash table (-w24)
    0.534	memory used for DFS stack (-m10000)
  128.730	total actual memory usage



pan: elapsed time 0 seconds
spin: prodcons1.pml:10, Error: assertion violated
spin: text of failed assertion: assert((n<5))
spin: trail ends after 182 steps
#processes: 6
		buffer[0] = 0
		buffer[1] = 0
		b

rm: cannot remove '*.trail': No such file or directory
rm: cannot remove 'pan.*': No such file or directory


In [ ]:
%%file prodcons2.pml
#define wait(s) atomic { s > 0 -> s-- }
#define signal(s) s++

int buffer[5]
int n = 0

byte notEmpty = 0         // semaphores
byte notFull = 5

inline append(d) {
    assert n < 5
    atomic {
        buffer[n] = d
        n++
    }
}

inline take(d) {
    assert n > 0
    atomic {
        n--
        d = buffer[n]
    }
}

active proctype Producer() {
    int d
    do
    ::
      // produce
      wait(notFull)
      append(d)
      signal(notEmpty)
    od
}

active[5] proctype Consumer() {
    int d
    do
    ::
      wait(notEmpty)
      take(d)
      signal(notFull)
      // consume
    od
}


Writing prodcons2.pml


In [ ]:
%%bash
rm *.trail pan.*
FILE=prodcons2.pml
spin -a $FILE && gcc pan.c && ./a.out
[ -f $FILE.trail ] && spin -t $FILE


(Spin Version 6.4.6 -- 2 December 2016)
	+ Partial Order Reduction

Full statespace search for:
	never claim         	- (none specified)
	assertion violations	+
	acceptance   cycles 	- (not selected)
	invalid end states	+

State-vector 88 byte, depth reached 4363, errors: 0
     6144 states, stored
    24577 states, matched
    30721 transitions (= stored+matched)
        0 atomic steps
hash conflicts:         0 (resolved)

Stats on memory usage (in Megabytes):
    0.680	equivalent memory usage for states (stored*(State-vector + overhead))
    0.876	actual memory usage for states
  128.000	memory used for hash table (-w24)
    0.534	memory used for DFS stack (-m10000)
  129.315	total actual memory usage


unreached in proctype Producer
	prodcons2.pml:35, state 13, "-end-"
	(1 of 13 states)
unreached in proctype Consumer
	prodcons2.pml:46, state 13, "-end-"
	(1 of 13 states)

pan: elapsed time 0.02 seconds
pan: rate    307200 states/second
